In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import re
import time


In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.NavySchedule_db
collection = db.items

In [4]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
res = requests.get('https://navysports.com/schedule.aspx?path=wlax', headers=headers)
soup = bs(res.content, 'html.parser')
soup.find_all('span', class_='sidearm-schedule-game-opponent-name')

[<span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#16 Virginia on February 9 1:00 p.m." href="http://virginiasports.com/" target="_blank">#16 Virginia</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="Saint Joseph's on February 17 1:00 p.m." href="http://www.sjuhawks.com/" target="_blank">Saint Joseph's</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#19 Towson on February 23 3:00 p.m." href="http://www.towsontigers.com/" target="_blank">#19 Towson</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="Villanova on February 27 3:00 p.m." href="http://www.villanova.com/" target="_blank">Villanova</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#11 Florida on March 2 11:00 a.m." href="http://www.floridagators.com/" target="_blank">#11 Florida</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#1 Boston College on March 

In [5]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('span', class_='sidearm-schedule-game-opponent-name')

opponent = []

# Loop over td elements
for result in results:
            stuff = (result.a.text)
            clean_stuff= re.sub("\d", "",stuff)
            clean_stuff= re.sub("# ", "",clean_stuff)
            opponent.append(clean_stuff)

print(opponent)


['Virginia', "Saint Joseph's", 'Towson', 'Villanova', 'Florida', 'Boston College', 'Brown', 'Colgate', 'Bucknell', 'Lehigh', 'Holy Cross', 'Loyola', 'Georgetown', 'Army West Point', 'American', 'Lafayette', 'Boston University', 'Army West Point', 'Loyola', 'High Point', 'Virginia']


In [6]:
results2 = soup.find_all('span', class_='sidearm-schedule-game-conference-vs flex flex-inline')                                
  
home_away = []

# Loop over elements
for result2 in results2:
    if (result2.text):
            stuff = (result2.text)
            clean_stuff= re.sub("\n", "",stuff)
            home_away.append(clean_stuff)
            
print(home_away)

['at', 'vs', 'vs', 'at', 'vs', 'vs', 'vs', 'at', 'vs', 'at', 'vs', 'at', 'at', 'vs', 'vs', 'vs', 'at', 'vs', 'at', 'vs', 'at']


In [7]:
from datetime import datetime

results3 = soup.find_all('div', class_='sidearm-schedule-game-opponent-date flex-item-1')

period = []
day = []


# Loop over elements
for result3 in results3:
            stuff = ((result3.span.text)[0:3])
            clean_stuff = datetime.strptime(stuff,'%b')
            clean_stuff = clean_stuff.strftime('%m')
            period.append(clean_stuff)
            


            date_day = ((result3.span.text)[4:6])
            date_day = re.sub(" ", "",date_day)
            if len(date_day) == 1:
                date_day = "0" + date_day
            else:
                date_day = date_day
            day.append(date_day)   
            
print(period)          
print(day)   


['02', '02', '02', '02', '03', '03', '03', '03', '03', '03', '03', '04', '04', '04', '04', '04', '04', '05', '05', '05', '05']
['09', '17', '23', '27', '02', '06', '09', '16', '23', '27', '30', '06', '10', '13', '17', '20', '25', '02', '04', '10', '12']


In [8]:
import numpy as np
url_list = pd.DataFrame(np.column_stack([opponent,home_away,period,day]), columns=['key','Home_Away','Period','Day'])
url_list

,key,Home_Away,Period,Day
0,Virginia,at,02,09
1,Saint Joseph's,vs,02,17
2,Towson,vs,02,23
3,Villanova,at,02,27
4,Florida,vs,03,02
5,Boston College,vs,03,06
6,Brown,vs,03,09
7,Colgate,at,03,16
8,Bucknell,vs,03,23
9,Lehigh,at,03,27


In [9]:
# The path to our CSV file
file = "SystemSchool.csv"

# Read our Kickstarter data into pandas
df = pd.read_csv(file)
df.head()

,key,CONFERENCE,SystemName
0,Maryland,Big Ten,maryland
1,Boston College,ACC,boston-college
2,North Carolina,ACC,north-carolina
3,Northwestern,Big Ten,northwestern
4,Syracuse,ACC,syracuse


In [10]:
new_df = url_list.set_index('key').join(df.set_index('key'))

new_df = new_df.reset_index()

new_df = new_df.rename_axis("game_id")

new_df

,key,Home_Away,Period,Day,CONFERENCE,SystemName
game_id,,,,,,
0,American,vs,04,17,Patriot,american
1,Army West Point,vs,04,13,Patriot,army
2,Army West Point,vs,05,02,Patriot,army
3,Boston College,vs,03,06,ACC,boston-college
4,Boston University,at,04,25,Patriot,boston-u
5,Brown,vs,03,09,Ivy League,brown
6,Bucknell,vs,03,23,Patriot,bucknell
7,Colgate,at,03,16,Patriot,colgate
8,Florida,vs,03,02,AAC,florida


In [11]:
base = 'https://www.ncaa.com/game/lacrosse-women/d1/2019/'
p = '/play-by-play'
np = '-navy/play-by-play'

url_dict = []
    
for index, row in new_df.iterrows():
    if row['Home_Away'] == 'at':
        url = base + row['Period'] + '/' + row['Day'] + '/navy-' +  row['SystemName'] + p
        url_dict.append(url)
    else:
        url = base + row['Period'] + '/' + row['Day'] + '/' +  row['SystemName'] + np
        url_dict.append(url)

url_df = pd.DataFrame({"url_list": url_dict})
url_df




box_p = '/boxscore'
box_np = '-navy/boxscore'

box_url_dict = []
    
for index, row in new_df.iterrows():
    if row['Home_Away'] == 'at':
        url = base + row['Period'] + '/' + row['Day'] + '/navy-' +  row['SystemName'] + box_p
        box_url_dict.append(url)
    else:
        url = base + row['Period'] + '/' + row['Day'] + '/' +  row['SystemName'] + box_np
        box_url_dict.append(url)

box_url_df = pd.DataFrame({"url_list": box_url_dict})
box_url_df


,url_list
0,https://www.ncaa.com/game/lacrosse-women/d1/20...
1,https://www.ncaa.com/game/lacrosse-women/d1/20...
2,https://www.ncaa.com/game/lacrosse-women/d1/20...
3,https://www.ncaa.com/game/lacrosse-women/d1/20...
4,https://www.ncaa.com/game/lacrosse-women/d1/20...
5,https://www.ncaa.com/game/lacrosse-women/d1/20...
6,https://www.ncaa.com/game/lacrosse-women/d1/20...
7,https://www.ncaa.com/game/lacrosse-women/d1/20...
8,https://www.ncaa.com/game/lacrosse-women/d1/20...
9,https://www.ncaa.com/game/lacrosse-women/d1/20...


In [12]:
#import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine('sqlite:///urldb', echo=False)
url_df.to_sql(name='sample_table2', con=engine, if_exists = 'replace', index=False)

In [13]:
session = Session(engine)
test = session.execute("select * from sample_table2")

result_proxy = test.fetchall()

#create list of urls
game_urls = []



for row in result_proxy:
    game_url = dict(row.items())
    print(game_url)
    game_urls.append(game_url["url_list"])
   





{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/17/american-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/13/army-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05/02/army-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/06/boston-college-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/25/navy-boston-u/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/09/brown-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/23/bucknell-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/16/navy-colgate/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/02/florida-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/10/navy-georgetown/play-by-play'}
{'url_list': 'https://www

In [14]:
#import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine('sqlite:///urldb', echo=False)
box_url_df.to_sql(name='sample_table3', con=engine, if_exists = 'replace', index=False)

In [15]:

session = Session(engine)
test = session.execute("select * from sample_table3")

result_proxy = test.fetchall()

#create list of urls
box_game_urls = []



for row in result_proxy:
    box_game_url = dict(row.items())
    print(box_game_url)
    box_game_urls.append(box_game_url["url_list"])


{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/17/american-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/13/army-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05/02/army-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/06/boston-college-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/25/navy-boston-u/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/09/brown-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/23/bucknell-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/16/navy-colgate/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/02/florida-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/10/navy-georgetown/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05

In [16]:
## Now time for the web scraping of the box scores and play by play

In [17]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [18]:
print("Number of box score urls")
print(len(box_game_urls))
print("Number of play by play urls")
print(len(game_urls))

Number of box score urls
21
Number of play by play urls
21


In [19]:
game_id = 0

season_stats_df = pd.DataFrame(columns=['No', 'Name', 'POS', 'G', 'A', 'SH', 'SOG', 'GB','game_id', 'team'])
season_goalies_stats_df = pd.DataFrame(columns=['Goalies', 'POS', 'MIN', 'SAVES', 'GA', 'game_id', 'team'])
# Go to NCAA Boxscore through splinter module

for i in range(len(box_game_urls)):
        try:
            score_url = box_game_urls[i]
            browser.visit(score_url)

            html = browser.html
            soup = BeautifulSoup(html, 'lxml')
        
        
            away_tables = pd.read_html(html)
            away_team_selector = soup.find("div", {"class": "boxscore-team-selector-team awayTeam-bg-primary_color homeTeam-border-primary_color away active"})
            away_box_table = away_tables[1]
            away_goalies_box_table = away_tables[2]
            away_box_table['game_id'] = game_id
            away_box_table['team'] = away_team_selector.text
            away_goalies_box_table['game_id'] = game_id
            away_goalies_box_table['team'] = away_team_selector.text
        
        
            team_selectors = soup.find("div", {"class": "boxscore-team-selector"})
            xpath = '//div[@class="boxscore-team-selector-team homeTeam-bg-primary_color awayTeam-border-primary_color home"]'
            browser.find_by_xpath(xpath).click()
            html = browser.html
        
            home_tables = pd.read_html(html)
            home_team_selector = soup.find("div", {"class": "boxscore-team-selector-team homeTeam-bg-primary_color awayTeam-border-primary_color home"})
            home_box_table = home_tables[1]
            home_goalies_box_table = home_tables[2]
            home_box_table['game_id'] = game_id
            home_box_table['team'] = home_team_selector.text
            home_goalies_box_table['game_id'] = game_id
            home_goalies_box_table['team'] = home_team_selector.text

            season_stats_df = season_stats_df.append(away_box_table, sort=True)
            season_stats_df = season_stats_df.append(home_box_table, sort=True)
            season_goalies_stats_df = season_goalies_stats_df.append(away_goalies_box_table, sort=True)
            season_goalies_stats_df = season_goalies_stats_df.append(home_goalies_box_table, sort=True)
            game_id = game_id+1
         
        except:
            print(f"box score not retrievable for: {score_url}")
            pass



  

box score not retrievable for: https://www.ncaa.com/game/lacrosse-women/d1/2019/04/17/american-navy/boxscore
box score not retrievable for: https://www.ncaa.com/game/lacrosse-women/d1/2019/03/09/brown-navy/boxscore
box score not retrievable for: https://www.ncaa.com/game/lacrosse-women/d1/2019/03/16/navy-colgate/boxscore
box score not retrievable for: https://www.ncaa.com/game/lacrosse-women/d1/2019/03/27/navy-lehigh/boxscore
box score not retrievable for: https://www.ncaa.com/game/lacrosse-women/d1/2019/05/04/navy-loyola-maryland/boxscore
box score not retrievable for: https://www.ncaa.com/game/lacrosse-women/d1/2019/02/09/navy-virginia/boxscore


In [20]:
season_stats_df = season_stats_df.replace('\n','', regex=True)
season_goalies_stats_df = season_goalies_stats_df.replace('\n','', regex=True)

In [21]:
season_stats_df.head(100)

,A,G,GB,Name,No,POS,SH,SOG,game_id,team
0,0,0,0,CHARLOTTE SPENCER,2,M,0,0,0,Army
1,0,0,2,SAMANTHA STEWART,10,M,2,1,0,Army
2,0,0,0,MANUELA CORTES,13,M,1,1,0,Army
3,0,0,1,LAUREN BREDENBURG,23,M,0,0,0,Army
4,1,1,0,RILEE SCOTT,44,M,1,1,0,Army
5,0,0,1,MADDIE BURNS,42,GK,0,0,0,Army
6,0,0,1,NIA CRUMP,1,D,0,0,0,Army
7,0,0,1,TAYLOR KORPELA,35,D,0,0,0,Army
8,1,0,0,OLIVIA CARTER,18,A,0,0,0,Army
9,3,2,0,CAROLINE RAYMOND,21,A,3,2,0,Army


In [22]:
season_goalies_stats_df.head(50)

,GA,Goalies,MIN,POS,SAVES,game_id,team
0,12,MADDIE BURNS,NaN,GK,5,0,Army
1,9,HANNAH SLOMKOWSKI,NaN,NaN,0,0,Army
2,21,Total,NaN,Total,5,0,Army
0,12,MADDIE BURNS,NaN,GK,5,0,Navy
1,9,HANNAH SLOMKOWSKI,NaN,NaN,0,0,Navy
2,21,Total,NaN,Total,5,0,Navy
0,20,MADDIE BURNS,NaN,G,13,1,Army
1,20,Total,NaN,Total,13,1,Army
0,10,GAB HARCHELROAD,NaN,G,6,1,Navy
1,0,ANNIE MALONEY,NaN,NaN,0,1,Navy


In [23]:
game_id = 0
season_play_by_play_df = pd.DataFrame(columns=['description', 'score', 'team', 'time','game_id'])

# Go to NCAA Play by play  through splinter module

for i in range(len(box_game_urls)):
    score_url = game_urls[i]
    browser.visit(score_url)
    html = browser.html
    #response = requests.get(url)
    soup = BeautifulSoup(html, 'lxml')
    all_rows = soup.find_all('tr')
    df_data = []
    for row in all_rows:
        try:
            img = row.find(class_="team")
            df_data.append({
                "time": row.find(class_="time").text,
                "team": img.find("img")["title"],
                "score": row.find(class_="score").text,
                "description": row.find(class_="description").text,
                "game_id": game_id
            })
            #print(row.find(class_="time").text)
            #print(img.find("img")["title"])
            #print(row.find(class_="score").text)
            #print(row.find(class_="description").text)
        except:
            #print(f"play by play not found {score_url}")
            pass
    game_id = game_id+1
    play_table_df = pd.DataFrame(df_data)
    season_play_by_play_df = season_play_by_play_df.append(play_table_df, sort=True)

In [24]:
season_play_by_play_df = season_play_by_play_df.replace('\n','', regex=True)

In [25]:
#join with schedule data for more complete dataset
season_play_by_play_df = season_play_by_play_df.join(new_df, on='game_id')

In [26]:
season_play_by_play_df.head(300)

,description,game_id,score,team,time,key,Home_Away,Period,Day,CONFERENCE,SystemName
0,"Geier, Fiona at goalie for AU.",0,,American,30:00,American,vs,04,17,Patriot,american
1,"Harchelroad, Gab at goalie for NAVY.",0,,Navy,30:00,American,vs,04,17,Patriot,american
2,"Draw control by AU Goldblum, Kendall.",0,,American,00:00,American,vs,04,17,Patriot,american
3,"Turnover by AU Jade, Izabella (caused by Heywa...",0,,American,29:26,American,vs,04,17,Patriot,american
4,"Ground ball pickup by NAVY Heyward, Annalise.",0,,Navy,29:23,American,vs,04,17,Patriot,american
5,Clear attempt by NAVY good.,0,,Navy,29:09,American,vs,04,17,Patriot,american
6,"GOAL by NAVY Victory, Nicole (FIRST GOAL), Ass...",0,0-1,Navy,28:56,American,vs,04,17,Patriot,american
7,"Draw control by NAVY Roelofs, Reagan.",0,,Navy,00:00,American,vs,04,17,Patriot,american
8,"GOAL by NAVY O'Sullivan, Andie, Assist by Heyw...",0,0-2,Navy,28:29,American,vs,04,17,Patriot,american
9,"Draw control by NAVY Harris, Kayla.",0,,Navy,00:00,American,vs,04,17,Patriot,american


In [27]:
#test sample recall of play by play for single game by game id
print(season_play_by_play_df.loc[season_play_by_play_df['game_id'] ==0])

                                           description  game_id  score  \
0                       Geier, Fiona at goalie for AU.        0          
1                 Harchelroad, Gab at goalie for NAVY.        0          
2                Draw control by AU Goldblum, Kendall.        0          
3    Turnover by AU Jade, Izabella (caused by Heywa...        0          
4        Ground ball pickup by NAVY Heyward, Annalise.        0          
5                          Clear attempt by NAVY good.        0          
6    GOAL by NAVY Victory, Nicole (FIRST GOAL), Ass...        0    0-1   
7                Draw control by NAVY Roelofs, Reagan.        0          
8    GOAL by NAVY O'Sullivan, Andie, Assist by Heyw...        0    0-2   
9                  Draw control by NAVY Harris, Kayla.        0          
10                           Foul on AU Brennan, Leah.        0          
11      Shot by NAVY Harris, Kayla, SAVE Geier, Fiona.        0          
12   GOAL by NAVY Larkin, Kelly, Assis

In [28]:
season_stats_df = season_stats_df.join(new_df, on='game_id')
season_goalies_stats_df = season_goalies_stats_df.join(new_df, on='game_id')

In [29]:
#test sample recall of box score for single game by game id

print(season_stats_df.loc[season_stats_df['game_id'] ==6])


    A  G  GB                  Name  No  POS  SH  SOG  game_id        team  \
0   0  0   1         NATALIE IRWIN   5    M   0    0        6        Navy   
1   2  3   3      ANNALISE HEYWARD  14    M   6    4        6        Navy   
2   0  0   1          KAYLA HARRIS  25    M   4    3        6        Navy   
3   0  0   1       GAB HARCHELROAD  39   GK   0    0        6        Navy   
4   0  0   4               GIL EBY   2    D   0    0        6        Navy   
5   0  0   2         MARIE VALENTI   8    D   0    0        6        Navy   
6   0  0   2         CAROLINE KWON  13    D   0    0        6        Navy   
7   0  0   2          ANNE RICHTER  19    D   0    0        6        Navy   
8   6  4   1          KELLY LARKIN   7    A   6    4        6        Navy   
9   0  0   1      ANDIE O'SULLIVAN  30    A   1    1        6        Navy   
10  1  5   0        NICOLE VICTORY  44    A   5    5        6        Navy   
11  0  0   1        REAGAN ROELOFS  50    A   1    1        6        Navy   

In [30]:
#test sample recall of goalies box score for single game by game id

print(season_goalies_stats_df.loc[season_goalies_stats_df['game_id'] ==7])

   GA     Goalies  MIN    POS  SAVES  game_id        team      key Home_Away  \
0  10  SARAH ZETO  NaN     GK      1        7  High Point  Colgate        at   
1   6   JILL RALL  NaN    NaN      7        7  High Point  Colgate        at   
2  16       Total  NaN  Total      8        7  High Point  Colgate        at   
0  10  SARAH ZETO  NaN     GK      1        7        Navy  Colgate        at   
1   6   JILL RALL  NaN    NaN      7        7        Navy  Colgate        at   
2  16       Total  NaN  Total      8        7        Navy  Colgate        at   

  Period Day CONFERENCE SystemName  
0     03  16    Patriot    colgate  
1     03  16    Patriot    colgate  
2     03  16    Patriot    colgate  
0     03  16    Patriot    colgate  
1     03  16    Patriot    colgate  
2     03  16    Patriot    colgate  


In [31]:
#dump all of the data into mySQL

#import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine('sqlite:///urldb', echo=False)


season_play_by_play_df.to_sql(name='season_play_by_play', con=engine, if_exists = 'replace', index=False)
season_stats_df.to_sql(name='season_stats', con=engine, if_exists = 'replace', index=False)
season_goalies_stats_df.to_sql(name='season_goalies_stats', con=engine, if_exists = 'replace', index=False)

In [32]:
#test the mySQL
session = Session(engine)
test = session.execute("select * from season_goalies_stats")

result_proxy = test.fetchall()

print(result_proxy)

[(12, 'MADDIE BURNS', None, 'GK', 5, 0, 'Army', 'American', 'vs', '04', '17', 'Patriot', 'american'), (9, 'HANNAH SLOMKOWSKI', None, None, 0, 0, 'Army', 'American', 'vs', '04', '17', 'Patriot', 'american'), (21, 'Total', None, 'Total', 5, 0, 'Army', 'American', 'vs', '04', '17', 'Patriot', 'american'), (12, 'MADDIE BURNS', None, 'GK', 5, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (9, 'HANNAH SLOMKOWSKI', None, None, 0, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (21, 'Total', None, 'Total', 5, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (20, 'MADDIE BURNS', None, 'G', 13, 1, 'Army', 'Army West Point', 'vs', '04', '13', 'Patriot', 'army'), (20, 'Total', None, 'Total', 13, 1, 'Army', 'Army West Point', 'vs', '04', '13', 'Patriot', 'army'), (10, 'GAB HARCHELROAD', None, 'G', 6, 1, 'Navy', 'Army West Point', 'vs', '04', '13', 'Patriot', 'army'), (0, 'ANNIE MALONEY', None, None, 0, 1, 'Navy', 'Army West Point', 'vs', '04', '13'

In [33]:
class Game():
    def __init__(self):
        self.shots = 0
        self.draw_controls = 0
        self.fouls = 0
        self.free_position_attempts = 0
        self.turnovers = 0
        self.ground_ball_pickups = 0
        self.clear_attempts = 0
        self.goals = 0
        self.timeouts = 0
        self.Navy_shots = 0
        self.opponent_shots = 0
        self.Navy_draw_controls = 0
        self.opponent_draw_controls = 0
        self.Navy_fouls = 0
        self.opponent_fouls = 0
        self.Navy_free_positions = 0
        self.opponent_free_positions = 0
        self.Navy_turnovers = 0
        self.opponent_turnovers = 0
        self.Navy_ground_ball_pickups = 0
        self.opponent_ground_ball_pickups = 0
        self.Navy_clear_attempts = 0
        self.Navy_successful_clears = 0
        self.Navy_failed_clears = 0
        self.opponent_clear_attempts = 0
        self.opponent_successful_clears = 0
        self.opponent_failed_clears = 0
        self.Navy_goals = 0
        self.opponent_goals = 0
        self.Navy_timeouts = 0
        self.opponent_timeouts = 0
        self.possessions = 0
        self.Navy_possessions = 0
        self.opponent_possessions = 0
    
    def print(self):
        attrs = vars(self)
        print(', '.join("%s: %s" % item for item in attrs.items()))

    def newPos(self):
        print("Adding new pos")
        
class Possession():
    team = "n/a"
    started_by = ""
    ended_with = ""

In [34]:

games = []


for i in range(len(box_game_urls)):
    
    play_by_play_df = season_play_by_play_df.loc[season_play_by_play_df['game_id'] == i]

    game = Game()

    for j in range(len(play_by_play_df)):
        current_row = play_by_play_df.iloc[j]
        current_description = current_row['description']
        print(current_description)
        list_of_words = current_description.split()



        if (current_description.find('Shot') != -1): 
                search_word = "by"
                game.shots = game.shots + 1

                shooting_team = list_of_words[list_of_words.index(search_word) + 1]

                if shooting_team == "NAVY":

                    game.Navy_shots = game.Navy_shots + 1

                else:

                    game.opponent_shots = game.opponent_shots + 1

        else: 
                game.shots = game.shots



        if (current_description.find('Draw control') != -1): 
                search_word = "by"
                game.draw_controls = game.draw_controls + 1
                game.possessions = game.possessions + 1
                draw_control_team = list_of_words[list_of_words.index(search_word) + 1]

                if draw_control_team == "NAVY":

                    game.Navy_draw_controls = game.Navy_draw_controls + 1
                    game.Navy_possessions = game.Navy_possessions + 1


                else:

                    game.opponent_draw_controls = game.opponent_draw_controls + 1
                    game.opponent_possessions = game.opponent_possessions + 1
        else: 
                game.draw_controls = game.draw_controls




        if (current_description.find('Foul') != -1): 
                search_word = "on"
                game.fouls = game.fouls + 1
                foul_on_team = list_of_words[list_of_words.index(search_word) + 1]

                if foul_on_team == "NAVY":

                    game.Navy_fouls = game.Navy_fouls + 1

                else:

                    game.opponent_fouls = game.opponent_fouls + 1

        else: 
                game.fouls = game.fouls





        if (current_description.find('Free position attempt') != -1): 
                search_word = "for"
                game.free_position_attempts = game.free_position_attempts + 1
                free_position_team = list_of_words[list_of_words.index(search_word) + 1]

                if free_position_team == "NAVY":

                    game.Navy_free_positions = game.Navy_free_positions + 1

                else:

                    game.opponent_free_positions = game.opponent_free_positions + 1

        else: 
                game.free_position_attempts = game.free_position_attempts



        if (current_description.find('Turnover') != -1): 
                search_word = "by"
                game.turnovers = game.turnovers + 1
                turnover_team = list_of_words[list_of_words.index(search_word) + 1]


                if turnover_team == "NAVY":

                    game.Navy_turnovers = game.Navy_turnovers + 1
                    game.opponent_possessions = game_1_opponent_possessions + 1
                    game.possessions = game.possessions + 1

                else:

                    game.opponent_turnovers = game.opponent_turnovers + 1
                    game.Navy_possessions = game.Navy_possessions + 1
                    game.possessions = game.possessions + 1
        else: 
                game.turnovers = game.turnovers



        if (current_description.find('Ground ball pickup') != -1): 
                search_word = "by"
                game.ground_ball_pickups = game.ground_ball_pickups + 1
                ground_ball_pickup_team = list_of_words[list_of_words.index(search_word) + 1]

                if ground_ball_pickup_team == "NAVY":

                    game.Navy_ground_ball_pickups = game.Navy_ground_ball_pickups + 1

                else:

                    game.opponent_ground_ball_pickups = game.opponent_ground_ball_pickups + 1   

        else: 
                game.ground_ball_pickups = game.ground_ball_pickups


        if (current_description.find('Clear attempt') != -1): 
                search_word = "by"
                game.clear_attempts = game.clear_attempts + 1
                clear_attempt_team = list_of_words[list_of_words.index(search_word) + 1]
                pass_or_fail_status = list_of_words[-1]
                #print(pass_or_fail_status)
                if clear_attempt_team == "NAVY":

                    game.Navy_clear_attempts = game.Navy_clear_attempts + 1

                    if pass_or_fail_status == "good.":

                        game.Navy_successful_clears = game.Navy_successful_clears + 1

                    else:
                        game.Navy_failed_clears = game.Navy_failed_clears +1

                else:

                    game.opponent_clear_attempts = game.opponent_clear_attempts + 1   

                    if pass_or_fail_status == "good.":

                        game.opponent_successful_clears = game.opponent_successful_clears + 1

                    else:
                        game.opponent_failed_clears = game.opponent_failed_clears + 1


        else: 
                game.clear_attempts = game.clear_attempts


        if (current_description.find('GOAL') != -1): 
                search_word = "by"
                game.goals = game.goals + 1
                scoring_team = list_of_words[list_of_words.index(search_word) + 1]

                if scoring_team == "NAVY":

                    game.Navy_goals = game.Navy_goals + 1

                else:

                    game.opponent_goals = game.opponent_goals + 1   



        else: 
                game.goals = game.goals



        if (current_description.find('Timeout') != -1): 
                search_word = "by"
                game.timeouts = game.timeouts + 1
                timeout_team = list_of_words[list_of_words.index(search_word) + 1]

                if timeout_team == "NAVY":

                    game.Navy_timeouts = game.Navy_timeouts + 1

                else:

                    game.opponent_timeouts = game.opponent_timeouts + 1   


        else: 
                game.timeouts = game.timeouts

    #games['game_{}'.format(i)] = game
    games.append(game)
            

Geier, Fiona at goalie for AU.
Harchelroad, Gab at goalie for NAVY.
Draw control by AU Goldblum, Kendall.
Turnover by AU Jade, Izabella (caused by Heyward, Annalise).
Ground ball pickup by NAVY Heyward, Annalise.
Clear attempt by NAVY good.
GOAL by NAVY Victory, Nicole (FIRST GOAL), Assist by Roelofs, Reagan, goal number 15 for season.
Draw control by NAVY Roelofs, Reagan.
GOAL by NAVY O'Sullivan, Andie, Assist by Heyward, Annalise, goal number 27 for season.
Draw control by NAVY Harris, Kayla.
Foul on AU Brennan, Leah.
Shot by NAVY Harris, Kayla, SAVE Geier, Fiona.
GOAL by NAVY Larkin, Kelly, Assist by Victory, Nicole, goal number 49 for season.
Draw control by NAVY Harris, Kayla.
GOAL by NAVY O'Sullivan, Andie, Assist by Victory, Nicole, goal number 28 for season.
Timeout by AU.
Draw control by NAVY Roelofs, Reagan.
Shot by NAVY Heyward, Annalise WIDE.
Foul on AU Aguilar-Gervase, C..
Free position attempt for NAVY.
GOAL by NAVY Heyward, Annalise (FPGOAL) {free position shot}, goal nu

NameError: name 'game_1_opponent_possessions' is not defined

In [ ]:
current_game = games[6]

current_game.print()

In [ ]:
print(current_game.opponent_goals)